<font color="#CA3532"><h1 align="left">Limpieza de datos</h1></font>

**Manuel Sánchez-Montañés**

## <font color="#CA3532">Limpieza de datos</font>

La siguiente etapa del preprocesado es la limpieza de datos, que consiste en las siguientes tareas (entre otras):

- Tratamiento de valores ausentes (detección, filtrado, imputación)
- Tratamiento de ouliers (o "valores atípicos")
- Eliminación de información irrelevante (por ejemplo, atributos constantes)
- Detección y eliminación de patrones duplicados
- Eliminación y/o arreglo de inconsistencias en los datos

Primero importaremos las librerías que necesitaremos:

In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

matplotlib.style.use('ggplot')

### <font color="#CA3532">Eliminación de missing values</font>

Primero empezaremos con la base de datos "small.csv":

In [3]:
# Carga de datos:
data = pd.read_csv("../datasets/small.csv", na_values = ["?", "none"], sep = ",")
data

,var1,var2,var3,var4,var5,var6
0,1,4.0,NaN,NaN,NaN,NaN
1,2,2.0,3.0,NaN,NaN,38.0
2,1,2.5,2.5,1.0,tc,39.0
3,2,3.5,1.5,0.0,tc,34.0
4,2,2.5,2.5,NaN,tc,39.0


In [4]:
data.shape[0] # número de registros

5

In [5]:
data.isnull().sum()

var1    0
var2    0
var3    1
var4    3
var5    2
var6    1
dtype: int64

In [6]:
# Count number of missing values:
100*data.isnull().mean()

var1     0.0
var2     0.0
var3    20.0
var4    60.0
var5    40.0
var6    20.0
dtype: float64

Para eliminar missing values usaremos el método <a href="http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.dropna.html#pandas-dataframe-dropna">pandas.DataFrame.dropna</a>

In [7]:
# Eliminar todas las filas que contengan missing values:
#data.dropna(inplace=True)
# data = data.dropna()
data.dropna()

,var1,var2,var3,var4,var5,var6
2,1,2.5,2.5,1.0,tc,39.0
3,2,3.5,1.5,0.0,tc,34.0


In [ ]:
data

In [8]:
# Eliminar todas las columnas que contengan missing values:
data.dropna(axis = 1)

,var1,var2
0,1,4.0
1,2,2.0
2,1,2.5
3,2,3.5
4,2,2.5


In [ ]:
#?data.dropna

In [9]:
# Eliminar todas las filas con menos de 4 NO missing values:
data.dropna(thresh = 4)

,var1,var2,var3,var4,var5,var6
1,2,2.0,3.0,NaN,NaN,38.0
2,1,2.5,2.5,1.0,tc,39.0
3,2,3.5,1.5,0.0,tc,34.0
4,2,2.5,2.5,NaN,tc,39.0


### <font color="#CA3532">Imputación de missing values</font>

Para imputar missing values usaremos el método <a href="http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.fillna.html#pandas-dataframe-fillna">pandas.DataFrame.fillna</a>.

In [ ]:
data

In [ ]:
# Imputación de missing values usando una constante general:
data.fillna(0)

In [ ]:
diccionario = {"madrid":[1,2,3], "paris":3, "berlin":"alemania"}
diccionario["paris"]

In [ ]:
# Imputación de missing values usando una constante diferente por cada columna:
data.fillna({'var3': 0, 'var4': 0, 'var5': 'tc', 'var6': 30.0})

In [ ]:
data

In [ ]:
data_num = data.loc[:, data.dtypes != object]

In [ ]:
data_num.mean()

In [ ]:
# Imputación de missing values usando el valor medio de cada columna (solo para
# atributos numéricos):
data.fillna(data_num.mean())

In [ ]:
lista = [1,2,3]
tupla = (1,2,3)
diccionario = {'spain':'madrid', 'france':'paris',
               0:'berlin' }

In [ ]:
diccionario['spain']

In [ ]:
data['var5'].mode()[0]

In [ ]:
data['var5'].mode().iloc[0]

In [ ]:
#?data.fillna

In [ ]:
# Imputación de missing values con la el valor medio de cada columna
# (para atributos numéricos) y con la moda de cada columna (para atributos
# categóricos):
data.fillna({'var4': data['var4'].median(),
             'var3': data['var3'].mean(),
             'var5': data['var5'].mode().iloc[0],
             'var6': data['var6'].mean()})

### <font color="#CA3532">Detección de outliers</font>

Usaremos el conjunto de datos *labor* para ilustrar el proceso de detección y filtrado de outliers (o "valores atípicos"). En las siguientes celdas cargamos los datos y realizamos un preprocesamiento para simplificar el conjunto de datos (solo conservamos los atributos numéricos e imputamos los missing values con la media del atributo).

In [ ]:
# Conjunto de datos Labor:
data = pd.read_csv("../datasets/labor.csv",
                   na_values = ["?"], sep = ",")
data[:10]

In [ ]:
# Mantener solo los atributos numéricos:
data = data.loc[:, data.dtypes != object]
data[:10]

In [ ]:
# Estadísticas de los datos:
data.describe()

In [ ]:
# Imputación de missing values con la media, in place:
data.fillna(data.mean(), inplace = True)
data[:20]

In [ ]:
media = data["shift_diff"].mean()
std   = data["shift_diff"].std()
print("{} +/- {}".format(media, 3*std))

In [ ]:
data["shift_diff"].hist()

La siguiente celda detecta como outliers a aquellos valores cuya distancia a la media es mayor que 3 veces la desviación estándar. Finalmente muestra la lista de todos los atributos que tienen algún outlier.

In [ ]:
3*data.std()

In [ ]:
import numpy as np
np.abs(data - data.mean()) > 3.0*data.std()

In [ ]:
import numpy as np
is_outlier = np.abs(data - data.mean()) > 3.0*data.std()
is_outlier.head(5)

In [ ]:
data.columns

In [ ]:
is_outlier["holidays"].any()

In [ ]:
# Detección de outliers:
import numpy as np
is_outlier = np.abs(data - data.mean()) > 3.0*data.std()
for var in data.columns:
    print(var, is_outlier[var].any())

In [ ]:
# otra forma de verlo:
import numpy as np

separacion = np.abs(data - data.mean()) / data.std()
is_outlier = separacion > 3
#data[is_outlier.any(axis=1)]
separacion[is_outlier.any(axis=1)]

Ahora miremos con cuidado a las variables *hours*, *stby_pay*, *shift_diff* y *holidays*, todas con outliers. Para cada una de ellas sacaremos por pantalla la media y desviación estándar, mostraremos un histograma, y sacaremos la lista de outliers.

In [ ]:
var = 'hours'
print(data[var].mean(), data[var].std())
data[var].hist()
plt.title(var)
data[is_outlier[var]]

In [ ]:
# Detección de outliers con percentiles:

def percentile_outlier_detector(x, threshold=95):
    diff = (100 - threshold) / 2
    (minval, maxval) = np.percentile(x, [diff, 100 - diff])
    return ((x < minval) | (x > maxval))

is_outlier2 = percentile_outlier_detector(data["hours"])
data[is_outlier2]

Otra forma: detección de outliers con intercuartiles.

<img src="../imagenes/IQR.png" align="center" style="float" width="500">

* Calcular el primer y tercer cuartil (Q1 y Q3)
* Calcular el IQR (Q3 - Q1)
* Considerar outlier todos los puntos que se salgan del rango $[Q1–1.5\cdot IQR, \,\,\, Q3+1.5 \cdot IQR]$

In [ ]:
def is_outlier_IQR(df):
    Q1=df.quantile(0.25)  #unique , duplicates, nan, outliers y transformación logaritmica
    Q3=df.quantile(0.75)
    IQR = Q3-Q1
    return (df<(Q1-1.5*IQR)) | (df>(Q3+1.5*IQR))

def transformacion_logaritmica(col):

In [ ]:
# Por intercuartiles:
data[is_outlier_IQR(data["hours"])]

In [ ]:
data[["hours"]].boxplot();

In [ ]:
data.boxplot(figsize=(10,5));

In [ ]:
var = 'stby_pay'
print(data[var].mean(), data[var].std())
data[var].hist()
plt.title(var)
data[is_outlier[var]]

In [ ]:
# Por intercuartiles:
data[is_outlier_IQR(data[var])]

In [ ]:
var = 'shift_diff'
print(data[var].mean(), data[var].std())
data[var].hist()
plt.title(var)
data[is_outlier[var]]

In [ ]:
# Por intercuartiles:
data[is_outlier_IQR(data[var])]

In [ ]:
var = 'holidays'
print(data[var].mean(), data[var].std())
data[var].hist()
plt.title(var)
data[is_outlier[var]]

In [ ]:
# Por intercuartiles:
data[is_outlier_IQR(data[var])]

### <font color="#CA3532">Filtrado de outliers</font>

La opción más sencilla es eliminar todas las filas que contienen algún valor atípico.

In [ ]:
is_outlier.any()

In [ ]:
is_outlier

In [ ]:
# Eliminación de todas las filas que contienen algún valor atípico:
data[~is_outlier.any(axis = 1)]

### <font color="#CA3532">Reemplazamiento de valores atípicos</font>

Otra opción es reemplazar los valores atípicos por otro valor. Una opción es la media más / menos tres veces la desviación estándar. En la celda siguiente hacemos esto para la variable *hours*.

In [ ]:
var = 'holidays'
var_mean = data[var].mean()
var_std  = data[var].std()
print(var_mean, var_std, "\n")
print(data[is_outlier[var]])
plt.figure()
data[var].hist()
plt.title(var)
data[var][is_outlier[var]] = data[var][is_outlier[var]].clip(var_mean-3*var_std,
                                                             var_mean+3*var_std)
print(data[is_outlier[var]])
plt.figure()
data[var].hist()
plt.title(var)

In [ ]:
11.094339622641511 + 3*1.2139685648936156

### <font color="#CA3532">Eliminación de atributos constantes</font>

Un atributo cuyo valor es constante para todas las instancias es completamente inútil para cualquier algoritmo de aprendizaje.

Volvamos a cargar el conjunto de datos *labor* y lo manipularemos un poco para que tenga atributos constantes e ilustrar este problema.

In [ ]:
# Conjunto de datos Labor:
data = pd.read_csv("../datasets/labor.csv", na_values = ["?"], sep = ",")
data[:10]

In [ ]:
# Pondremos valores constantes a los atributos cola y educ_allw:
data["cola"] = 'tc'
data["educ_allw"] = 'yes'
data[:10]

In [ ]:
# Número de valores únicos excluyendo NaN:
data.apply(pd.Series.nunique)

In [ ]:
# Lista de atributos con un solo valor:
data.apply(pd.Series.nunique) == 1

In [ ]:
data.drop(["cola", "educ_allw"], axis=1, inplace=True)
data

In [ ]:
# Otra manera de eliminar las columnas cola y educ_allw:

# del data["cola"]
# del data["educ_allw"]
# data

### <font color="#CA3532">Detección de duplicados</font>

Podemos usar el método <a href="http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.duplicated.html#pandas-dataframe-duplicated">pandas.DataFrame.duplicated</a> para detectar filas duplicadas en los datos, y el método <a href="http://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.drop_duplicates.html#pandas-dataframe-drop-duplicates">pandas.DataFrame.drop_duplicates</a> para eliminar todos los duplicados.

Cargaremos de nuevo la base de datos *labor* y la manipularemos un poco para que ahora tenga filas duplicadas.

In [ ]:
# Conjunto de datos Labor, mantendremos solo un subconjunto de los atributos:
data = pd.read_csv("../datasets/labor.csv", na_values = ["?"], sep = ",")
keep_cols = ["pension", "educ_allw", "holidays", "vacation"]
data = data[keep_cols]

In [ ]:
data

In [ ]:
aux = data.iloc[-1:0:-1]
aux = data
aux[aux.duplicated(keep = 'first')].sort_values(by = ["pension", "educ_allw",
                                   "holidays", "vacation"])

In [ ]:
# Detección de duplicados:
data[data.duplicated(keep = False)].sort_values(by = ["pension", "educ_allw",
                                                      "holidays", "vacation"])

In [ ]:
# Eliminación de duplicados:
data_nodup = data.drop_duplicates()
data_nodup.duplicated().any()

### <font color="#CA3532">Otras inconsistencias</font>

Podemos detectar muchos más problemas mirando detenidamente a los datos. Algunos ejemplos:

- ¿Las fechas parecen fechas?
- ¿La edad de una persona está en un rango razonable? 
- ¿Hay valores "extraños" que deberían ser considerados como NaN?
- ¿Hay atributos numéricos que deberían ser considerados categóricos?
- ¿Hay valores diferentes que se refieren a la misma cosa?


### <font color="#CA3532">Ejercicio</font>

Carga la base de datos *loan*, explora los datos usando los análisis previos e intenta responder a las cuestiones siguientes:

- ¿Hay missing values? Si es así, filtra o imputa estos valores usando la estrategia más apropiada. 
- ¿Hay outliers? 
- ¿Hay atributos constantes o instancias duplicadas?
- ¿Hay alguna otra inconsistencia en los datos que debería ser corregida?